# 02_07: Connecting Python to Neo4j

In [ ]:
import os
from dotenv import load_dotenv

from neo4j import GraphDatabase

In [ ]:
load_dotenv(dotenv_path='../.env')

URI = os.getenv("NEO4J_URI")
USER = os.getenv("NEO4J_USER")
PWD = os.getenv("NEO4J_PWD")

## A note about this next code block

There are many ways to connect to Neo4j from Python.  This here is a little helper class that manages the connection for you and includes some basic error handling.  We will use this class in pretty much every notebook that needs to connect to Neo4j in this course, but this is what it looks like.  For the following videos you can find this class in `/utils/neo4j_conn.py`.

In [ ]:
class Neo4jConnection:

    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

In [ ]:
neo_conn = Neo4jConnection(uri=URI, user=USER, pwd=PWD)

In [ ]:
neo_conn.query('MATCH (n) RETURN COUNT(n) AS total')

In [ ]:
neo_conn.query('MATCH(a:Employee)-[r:REPORTS_TO]-(b) RETURN a, r, b')